In [ ]:
import torch
import numpy as np
import math

#import pykeops
#pykeops.clean_pykeops()

In [ ]:
device = torch.device("cuda")

In [ ]:
from gechebnet.graph.graph import SE2GEGraph

In [ ]:
xi, eps = 1., 1.
se2_graph = SE2GEGraph(
    nx=10,
    ny=10,
    ntheta=6,
    knn=16,
    sigmas=(xi / eps, xi, 1.0),
    weight_kernel=lambda sqdistc, sigmac: torch.exp(-sqdistc / sigmac),
    kappa=0.,
    device=torch.device("cuda")
)

## ChebNet

In [ ]:
from gechebnet.model.chebnet import GEChebNet

In [ ]:
K = 5
in_channels = 3
out_channels = 10
hidden_channels = [16, 17, 19, 23]

In [ ]:
model = GEChebNet(se2_graph, K, in_channels, hidden_channels, out_channels, device=device)
model

In [ ]:
model = model.to(device)

In [ ]:
x = torch.rand(16, 3, 600, device=device)
x.shape

In [ ]:
y = model(x)
y.shape

## ResChebNet

In [ ]:
from gechebnet.model.reschebnet import ResGEChebNet

In [ ]:
K = 5
in_channels = 3
out_channels = 10
hidden_channels = [[32, 32, 32]]

In [ ]:
model = ResGEChebNet(se2_graph, K, in_channels, hidden_channels, out_channels, device=device)
model

In [ ]:
model = model.to(device)

In [ ]:
x = torch.rand(16, 3, 600, device=device)
x.shape

In [ ]:
y = model(x)
y.shape